# 18_Defining_One_to_One_Relationships

In this notebook, we'll explore how to define **one-to-one relationships** between entities in Spring Data JPA. We'll focus on a `User` having a **single profile**, and we'll explain every annotation and why it's placed where it is. 🧠✅

---

### 👤 `User` Entity

```java
@Entity
@Table(name = "users")
@Builder
@Setter
@Getter
@AllArgsConstructor
@NoArgsConstructor
@ToString
public class User {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    @Column(name="id")
    private Long id;

    @Column(name = "name")
    private String name;

    @Column(name = "email")
    private String email;

    @Column(name = "password")
    private String password;

    // 🏠 One-to-Many Relationship with Addresses (already defined)
    @OneToMany(mappedBy = "user")
    @Builder.Default
    private List<Address> addresses = new ArrayList<>();

    // 🔖 Many-to-Many Relationship with Tags (already defined)
    @ManyToMany
    @JoinTable(
        name = "user_tags",
        joinColumns = @JoinColumn(name = "user_id"),
        inverseJoinColumns = @JoinColumn(name = "tag_id")
    )
    @Builder.Default
    private Set<Tag> tags = new HashSet<>();

    // 🧬 One-to-One Relationship with Profile
    @OneToOne(mappedBy = "user")
    private Profile profile;
}
```

### 🧬 `Profile` Entity

```java
@Entity
@Table(name="Profiles")
@Builder
@Setter
@Getter
@AllArgsConstructor
@NoArgsConstructor
@ToString
public class Profile {

    @Id
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    @Column(name="bio")
    private String bio;

    @Column(name="phone_number")
    private String phobeNumber;

    @Column(name="date_of_birth")
    private String dateOfBirth;

    @Column(name="loyalty_points")
    private String loyaltyPoints;

    // 🧍‍♂️ One-to-One side owning the relationship
    @OneToOne
    @JoinColumn(name = "id") // references user.id
    @MapsId // Shares the primary key with User
    @ToString.Exclude
    private User user;
}
```

---

### 📘 Explanation of Key Annotations

#### 🔄 `@OneToOne`

Used on both `User` and `Profile` to indicate a one-to-one mapping between the two entities.

* In `User`, it's `mappedBy = "user"` to indicate the other side owns the relationship.
* In `Profile`, it's the owning side that contains the actual foreign key.

#### 🔗 `@JoinColumn(name = "id")`

* Indicates which column in the `Profile` table will be used to link to the `User` entity.
* Since we’re sharing primary keys, `id` serves as both PK and FK.

#### 🧬 `@MapsId`

* Ensures that the `Profile` shares the same ID as the `User`.
* Guarantees a tight coupling: the profile **cannot exist** without a user.

#### 🧍‍♂️ Why `mappedBy` is in `User`, not in `Profile`?

* Because `Profile` holds the foreign key (`user_id`), it is the **owning side**.
* The `User` just references it by `mappedBy` to keep bidirectional navigation.

#### 🙈 `@ToString.Exclude`

* Prevents infinite recursion when using Lombok’s `@ToString()` due to the bidirectional relationship.

---

### 🧪 Example Usage

```java
@SpringBootApplication
public class StoreApplication {

	public static void main(String[] args) {
		ApplicationContext context = SpringApplication.run(StoreApplication.class, args);

		var user = User.builder()
				.name("John")
				.password("1234")
				.email("john@gmail.com")
				.build();

		var profile = Profile.builder()
				.bio("bio")
				.build();

		// Setting both sides explicitly
		user.setProfile(profile);
		profile.setUser(user);

		System.out.println(user);
	}
}
```

---

🎉 Now you’ve learned how to create a one-to-one relationship in JPA with shared primary keys and clearly understood ownership! Let me know if you want to persist this using a repository/service layer too!
